In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
concentrations_df = pd.read_csv('419_TiO_H2S_Concentrations.txt', sep='\t')
concentrations_df = concentrations_df[1:]

In [3]:
concentrations = np.array(concentrations_df)[:, 1]
names = np.array(concentrations_df)[:, 0]

In [4]:
curve_dict = {}

In [5]:
for i in range(0, len(names)):
    curve_dict[names[i]] = int(concentrations[i])

In [6]:
d = 'Curves'
dfs_dict = {}

for path in os.listdir(d):
    dfs = []
    pixel_path = os.path.join(d, path)
    
    for file in sorted(os.listdir(pixel_path), key=lambda x: int(x.split(' ')[0])):
        concentrations_list = []
        concentrations_list.append(curve_dict[file[:-4]])

        curve_path = os.path.join(pixel_path, file)
        df = pd.read_csv(curve_path, sep='\t')
        
        df['SDAmpers'][1:] = df['SDAmpers'][1:].apply(float)
        df['TimeStamp'][1:] = df['TimeStamp'][1:].apply(float)*0.001
        
        df['Concentrations'] = concentrations_list*len(df) 
        dfs.append(df[['TimeStamp', 'SDAmpers', 'Concentrations']][1:])

    dfs_dict[path] = dfs

In [47]:
from plotly.offline import init_notebook_mode, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [72]:
traces = []
pixels = ['Pixel 1', 'Pixel 2', 'Pixel 3', 'Pixel 4', 'Pixel 5']

for N in [0, 25, 37, 47, 55, 66, 75, 115, 160]:
    for pixel in pixels:
        traces.append(
            go.Scatter(
                x=dfs_dict[pixel][N].TimeStamp,
                y=dfs_dict[pixel][N].SDAmpers, 
                name = 'P{}C{}'.format(pixel[-1], dfs_dict[pixel][N].Concentrations[1]),
                visible = (pixel == pixels[0]),
                mode = 'lines'
        ))

layout = go.Layout(
    title='..',
    updatemenus=list([
        dict(
            x=-0.05,
            y=1,
            yanchor='top',
            buttons=list([                
                dict(
                    args=['visible', [True] + [False]*4],
                    label='Pixel 1',
                    method='restyle',
                ),
                dict(
                    args=['visible', [False] + [True] + [False]*3],
                    label='Pixel 2',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*2 + [True] + [False]*2],
                    label='Pixel 3',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*3 + [True] + [False]],
                    label='Pixel 4',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*4 + [True]],
                    label='Pixel 5',
                    method='restyle'
                )
            ]),
        )
    ]),
)

fig = {'data': traces, 'layout': layout}
iplot(fig, show_link=False)